In [3]:
from datasets import load_dataset


ds = load_dataset("json", data_files="/home/ubuntu/alignment-handbook/test/iterative_dpo/iteration_0/candidates.json", split="train")
ds

Dataset({
    features: ['original', 'candidates'],
    num_rows: 100
})

In [12]:
s = ds[4]

In [13]:
s["original"]

[{'content': 'Q: In this task, you are given a sentence. You are expected to recognize the name of gene or protein. Although there might be several correct answers, you need to write one of them.\n A nonpeptidyl small molecule SB 247464 , capable of activating granulocyte - colony - stimulating factor ( G - CSF ) signal transduction pathways , was identified in a high - throughput assay in cultured cells .\nA:',
  'role': 'user'},
 {'content': 'G-CSF (granulocyte-colony stimulating factor)',
  'role': 'assistant'}]

In [17]:
import torch 
from transformers import pipeline
rm_pipe = pipeline(
    "sentiment-analysis",
    "sfairXC/FsfairX-LLaMA3-RM-v0.1",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.bfloat16},
    truncation=True,
    
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


UndefinedError: 'str object' has no attribute 'role'

In [21]:
s

{'original': [{'content': 'Q: In this task, you are given a sentence. You are expected to recognize the name of gene or protein. Although there might be several correct answers, you need to write one of them.\n A nonpeptidyl small molecule SB 247464 , capable of activating granulocyte - colony - stimulating factor ( G - CSF ) signal transduction pathways , was identified in a high - throughput assay in cultured cells .\nA:',
   'role': 'user'},
  {'content': 'G-CSF (granulocyte-colony stimulating factor)',
   'role': 'assistant'}],
 'candidates': [[{'content': 'Q: In this task, you are given a sentence. You are expected to recognize the name of gene or protein. Although there might be several correct answers, you need to write one of them.\n A nonpeptidyl small molecule SB 247464 , capable of activating granulocyte - colony - stimulating factor ( G - CSF ) signal transduction pathways , was identified in a high - throughput assay in cultured cells .\nA:',
    'role': 'user'},
   {'cont

In [25]:
[s["original"]] + s["candidates"]

[[{'content': 'Q: In this task, you are given a sentence. You are expected to recognize the name of gene or protein. Although there might be several correct answers, you need to write one of them.\n A nonpeptidyl small molecule SB 247464 , capable of activating granulocyte - colony - stimulating factor ( G - CSF ) signal transduction pathways , was identified in a high - throughput assay in cultured cells .\nA:',
   'role': 'user'},
  {'content': 'G-CSF (granulocyte-colony stimulating factor)',
   'role': 'assistant'}],
 [{'content': 'Q: In this task, you are given a sentence. You are expected to recognize the name of gene or protein. Although there might be several correct answers, you need to write one of them.\n A nonpeptidyl small molecule SB 247464 , capable of activating granulocyte - colony - stimulating factor ( G - CSF ) signal transduction pathways , was identified in a high - throughput assay in cultured cells .\nA:',
   'role': 'user'},
  {'content': 'In this task, the sent

In [33]:

pipe_kwargs = {
    "top_k": None,
    "function_to_apply": "none",
    "batch_size": 6
}

def format(s):
    return rm_pipe.tokenizer.apply_chat_template(s, tokenize=False, add_generation_prompt=False).replace(rm_pipe.tokenizer.bos_token, "")

batch = []
candidate_array = [s["original"]] + s["candidates"]
for c in [s["original"]] + s["candidates"]:
    batch.append(format(c))
res = rm_pipe(batch, **pipe_kwargs)
scores = [output[0]["score"] for output in res]
# convert to list with scores 
result = {
    "original": {
        "messages": s["original"],
        "score": scores[0]
        },
    "candidates": [
     {
         "messages": c,
         "score": scores[i+1]
     } for i, c in enumerate(s["candidates"])
    ]
}
print(result)

# # get the index of the two highest scores
# tensor = torch.tensor([output[0]["score"] for output in res])
# top2_values, top2_indices = torch.topk(tensor, 2)
# top2_indices.tolist()

# result = {
#     "chosen": candidate_array[top2_indices[0]],
#     "rejected": candidate_array[top2_indices[1]]
# }
# print(result["chosen"])
# print(result["rejected"])

# create new preference pairs from the two top candidates



{'original': {'messages': [{'content': 'Q: In this task, you are given a sentence. You are expected to recognize the name of gene or protein. Although there might be several correct answers, you need to write one of them.\n A nonpeptidyl small molecule SB 247464 , capable of activating granulocyte - colony - stimulating factor ( G - CSF ) signal transduction pathways , was identified in a high - throughput assay in cultured cells .\nA:', 'role': 'user'}, {'content': 'G-CSF (granulocyte-colony stimulating factor)', 'role': 'assistant'}], 'score': 0.46875}, 'candidates': [{'messages': [{'content': 'Q: In this task, you are given a sentence. You are expected to recognize the name of gene or protein. Although there might be several correct answers, you need to write one of them.\n A nonpeptidyl small molecule SB 247464 , capable of activating granulocyte - colony - stimulating factor ( G - CSF ) signal transduction pathways , was identified in a high - throughput assay in cultured cells .\

In [35]:
from datasets import Dataset

x = Dataset.from_list([result])

In [42]:
def create_pair(s):
    arr = [s["original"]] + s["candidates"]
    tensor = torch.tensor([s["score"] for s in arr])
    _, top2_indices = torch.topk(tensor, 2)
    top2_indices.tolist()

    return  {
        "chosen": arr[top2_indices[0]]["messages"],
        "rejected": arr[top2_indices[1]]["messages"]
    }

y = x.map(create_pair, remove_columns=x.features)
y[0]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

{'chosen': [{'content': 'Q: In this task, you are given a sentence. You are expected to recognize the name of gene or protein. Although there might be several correct answers, you need to write one of them.\n A nonpeptidyl small molecule SB 247464 , capable of activating granulocyte - colony - stimulating factor ( G - CSF ) signal transduction pathways , was identified in a high - throughput assay in cultured cells .\nA:',
   'role': 'user'},
  {'content': 'G-CSF (granulocyte-colony stimulating factor)',
   'role': 'assistant'}],
 'rejected': [{'content': 'Q: In this task, you are given a sentence. You are expected to recognize the name of gene or protein. Although there might be several correct answers, you need to write one of them.\n A nonpeptidyl small molecule SB 247464 , capable of activating granulocyte - colony - stimulating factor ( G - CSF ) signal transduction pathways , was identified in a high - throughput assay in cultured cells .\nA:',
   'role': 'user'},
  {'content': '

In [8]:
ds = load_dataset("json", data_files="//home/ubuntu/alignment-handbook/test/iterative_dpo/iteration_0/prompts.json", split="train")

ds[43]

# ds.to_json("test.json")

{'chosen': [{'content': 'This task is to translate a news commentary given in Arabic language into Czech language. Translate the input statement into the output language  while preserving the numberical values, special characters and context of the commentary done.\n\n[Q]: إن المشكلة الحقيقية في هذا التنظير المحموم للمؤامرات هي أنه يؤدي إلى تهييج مشاعر المواطنين ولكن بدون منحهم أساساً صلباً من الأدلة التي تمكنهم من بناء نظرتهم إلى العالم، وبدون أي نوع من الإرشاد البنَّاء الذي من ش��نه أن يسمح لهم بتوجيه مشاعرهم. وهذا هو السبب الذي يجعل الكثير من خيوط المناقشة تتحول من مجرد تكهنات مسلية من جانب المواطنين إلى خطاب الكراهية وجنون العظمة. وفي هذه البيئة المحمومة، وفي غياب المصادقة التحريرية أو الأدوات اللازمة للتعرف على المصادر، يتحول المواطنين إلى فريسة لزعماء الدهماء والغوغاء، كما رأينا أثناء الأسابيع الأخيرة في إطار الاجتماعات الحاشدة التي نظمتها سارة بالين بعد أن اتهمت نظريات الإنترنت باراك أوباما بأنه إرهابي أو على صلة بالإرهابيين.\n[A]: Skutečný problém tohoto horečnatého „konspiračn

In [33]:
from typing import Dict, List
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer


class ArmoRMPipeline:
    def __init__(self, model_id, device_map="auto", torch_dtype=torch.bfloat16, truncation=True, trust_remote_code=False, max_length=4096):
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_id,
            device_map=device_map,
            trust_remote_code=trust_remote_code,
            torch_dtype=torch_dtype,
        )
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_id,
            use_fast=True,
        )
        self.truncation = truncation
        self.device = self.model.device
        self.max_length = max_length

    def __call__(self, messages: List[Dict[str, str]]) -> Dict[str, float]:
        """
        messages: OpenAI chat messages to be scored
        Note: no batching since due to length differences, the model will have to pad to the max length which is not efficient
        Returns: a dictionary with the score between 0 and 1
        """
        input_ids = self.tokenizer.apply_chat_template(
            messages,
            return_tensors="pt",
            padding=True,
            truncation=self.truncation,
            max_length=self.max_length,
        ).to(self.device)
        with torch.no_grad():
            output = self.model(input_ids)
            score = output.score.float().item()
        return {"score": score}


In [34]:
# Create Reward Model Pipeline 
prompt = 'What are some synonyms for the word "beautiful"?'
rm = ArmoRMPipeline("RLHFlow/ArmoRM-Llama3-8B-v0.1", trust_remote_code=True)
# score the messages
response1 = 'Nicely, Beautifully, Handsome, Stunning, Wonderful, Gorgeous, Pretty, Stunning, Elegant'
score1 = rm([{"role": "user", "content": prompt}, {"role": "assistant", "content": response1}])
print(score1)

response2 = '''Certainly! Here are some synonyms for the word "beautiful":

1. Gorgeous
2. Lovely
3. Stunning
4. Attractive
5. Pretty
6. Elegant
7. Exquisite
8. Handsome
9. Charming
10. Alluring
11. Radiant
12. Magnificent
13. Graceful
14. Enchanting
15. Dazzling

These synonyms can be used in various contexts to convey the idea of beauty.'''
score2 = rm([{"role": "user", "content": prompt}, {"role": "assistant", "content": response2}])
print(score2)

response3 = 'Sorry i cannot answer this.'
score3 = rm([{"role": "user", "content": prompt}, {"role": "assistant", "content": response3}])
print(score3)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'score': 0.109375}
{'score': 0.1669921875}
{'score': 0.03759765625}


In [32]:
for i in [input, inputs2, inputs3]:
    print(rm(i))

{'score': 0.109375}
{'score': 0.1669921875}
{'score': 0.03759765625}
